In [ ]:
#例题17.10
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy
import scipy.stats as stats
import scipy.optimize as opt
import matplotlib.pyplot as plt
def poisson_logprob(mu, sign=-1):
    return np.sum(sign*stats.poisson.logpmf(y_obs, mu=mu))
# 读取数据文件
messages = pd.read_csv('data/QQ_data.csv')
with pm.Model() as model:
    # 创建一个概率模型
    mu = pm.Uniform('mu', lower=0, upper=60)
    likelihood = pm.Poisson('likelihood', mu=mu, observed=messages['numbers'].values)
    start = pm.find_MAP()
    step = pm.Metropolis()
    trace = pm.sample(20000, step, start=start, progressbar=True)
y_obs = messages['numbers'].values
#极大似然估计求解mu
freq_results = opt.minimize_scalar(poisson_logprob)
#traceplot函数来绘制后验采样的趋势图
pm.traceplot(trace, var_names=['mu'], lines={'mu': freq_results['x']})

In [ ]:
# 例17.11 定义一个抛硬币问题，假设硬币正面向上的概率为0.35，生成数据样本
import numpy as np
import scipy.stats as stats
import pymc3 as pm
np.random.seed(1)
n_experiments = 100 # 试验次数
theta_real = 0.35 # 硬币正面向上的概率参数 θ，用 theta_real 来表示
data = stats.bernoulli.rvs(p=theta_real, size=n_experiments)
print(data)

In [ ]:
with pm.Model() as our_first_model: # 构建了一个模型的容器
    theta = pm.Beta('theta', alpha=1, beta=1) # 指定先验
    #用跟先验相同的语法描述了似然概率，用 observed 参数传递了观测到的数据
    y= pm.Bernoulli('y', p=theta, observed=data)
    # 返回最大后验（Maximum a Posteriori，MAP），为采样方法提供一个初始点
    start=pm.find_MAP()
    # 采样方法Metropolis-Hastings 算法，PyMC3会根据不同参数的特性自动地赋予一个采样器
    step = pm.Metropolis()
    # 执行采样，其中第 1 个参数是采样次数，第 2 个和第 3 个参数分别是采样方法和初始点
    trace = pm.sample(1000, step=step, start=start)
    

In [ ]:
#样本路径图
burnin = 100
chain = trace[burnin:]
pm.traceplot(chain, lines={'theta': theta_real})

In [ ]:
#Gelman-Rubin 检验
pm.gelman_rubin(chain)

In [ ]:
pm.summary(chain)

In [ ]:
pm.autocorrplot(chain)

In [ ]:
pm.effective_n(chain)['theta']

In [ ]:
pm.plot_posterior(chain)